In [1]:
# importing necessary libraries
import numpy as np
import pandas as pd

In [2]:
# read the train and test sets
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train.head()




,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
# filling location and keyword with unknown and none
train['location'].fillna('Unknown', inplace=True)
test['location'].fillna('Unknown', inplace=True)
train['keyword'].fillna('None', inplace=True)
test['keyword'].fillna('None', inplace=True)


In [4]:
# separate targe and features

X = train[['text', 'location', 'keyword']]
y = train['target']



In [5]:
#train validaztion split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)



In [6]:
# combine all features as a single string
X_train['all'] = X_train['text'] + ' ' + X_train['location'] + ' ' + X_train['keyword']
X_val['all'] = X_val['text'] + ' ' + X_val['location'] + ' ' + X_val['keyword']
test['all'] = test['text'] + ' ' + test['location'] + ' ' + test['keyword']



<ipython-input-6-22b434eb261c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['all'] = X_train['text'] + ' ' + X_train['location'] + ' ' + X_train['keyword']
<ipython-input-6-22b434eb261c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['all'] = X_val['text'] + ' ' + X_val['location'] + ' ' + X_val['keyword']
